In [52]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import seawater as sea

# Open the NetCDF file
ds_temp = nc.Dataset('IonianTemperature.nc', 'r')  # Replace 'path_to_your_file.nc' with the actual file path


# Validate This:
# P = P_0 + ρgh // pressure, P_0 in hPa
# def findPressure(depth):
#     P_0 = 1013.25
#     density = 1025 # can slightly vary (kg/m^3)
#     g = 9.81 # m/s^2
#     h = depth # m
#     return P_0 + density*g*h

 
temp = ds_temp.variables['thetao']
depth = ds_temp.variables['depth']
lat = ds_temp.variables['latitude']
long = ds_temp.variables['longitude']
time = ds_temp.variables['time']

depths = depth[:]
times = time[:]


latitude,longitude = ds_temp.variables['latitude'], ds_temp.variables['longitude']

# extract lat/lon values (in degrees) to numpy arrays
latvals = latitude[:]; lonvals = longitude[:]

def get_closest_ij(lats, lons, latpt, lonpt):
    # Create 2D grid arrays for lats and lons
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    
    # Calculate squared distance from every grid point to the given point
    # Notice that we are using lat_grid and lon_grid which are 2D arrays
    dist_sq = (lat_grid - latpt)**2 + (lon_grid - lonpt)**2
    
    # Find the 1D index of the minimum squared distance
    minindex_flattened = dist_sq.argmin()
    
    # Convert the 1D index back to 2D indices for lat and lon arrays
    ij_min = np.unravel_index(minindex_flattened, dist_sq.shape)
    
    return ij_min

iy_min, ix_min = get_closest_ij(latvals, lonvals, 50., -140)
print(iy_min)
print(ix_min)


96
0


In [53]:
import numpy as np

def find_closest_index(array, target_value):
    """
    Finds the index of the closest value to the target_value in an array.

    Parameters:
    - array: np.ndarray, the array to search within.
    - target_value: float or int, the value to find the closest to in array.

    Returns:
    - tuple or int: The index of the closest value in the array. Returns a tuple for multidimensional arrays and an int for 1D arrays.
    """
    # Calculate the absolute differences from the target value
    absolute_differences = np.abs(array - target_value)
    
    # Find the 1D index of the element with the minimum absolute difference
    min_index_flattened = np.argmin(absolute_differences)
    
    # Convert the 1D index back to an index tuple for multidimensional arrays
    return np.unravel_index(min_index_flattened, array.shape)
def findex(masked_array, target_value):
    """
    Finds the index of the closest value to the target_value in a masked array.

    Parameters:
    - masked_array: np.ma.MaskedArray, the array to search within.
    - target_value: float or int, the value to find the closest to in masked_array.

    Returns:
    - tuple: The index of the closest value in the masked_array.
    """
    # Ensure the input is a masked array
    
    if not isinstance(masked_array, np.ma.MaskedArray):
        return find_closest_index(masked_array,target_value)

    # Compute the absolute difference with the target value, ignoring masked values
    absolute_differences = np.ma.abs(masked_array - target_value)

    # Find the index of the minimum difference (closest value)
    closest_index = np.ma.argmin(absolute_differences)

    # np.ma.argmin returns a flattened index if the array is multidimensional,
    # so you might need to convert it to a multidimensional index depending on your needs
    if masked_array.ndim > 1:
        closest_index = np.unravel_index(closest_index, masked_array.shape)

    return closest_index


In [54]:
ds_temp = nc.Dataset('IonianSalinity.nc', 'r')
ds_horiz= nc.Dataset('IonianHorizontal.nc', 'r')
ds_vertical= nc.Dataset('IonianVertical.nc', 'r')


east_vel = ds_horiz['uo']
north_vel = ds_vertical['wo']
sal = ds_temp['so']


''' TEST '''
# time, depth
closestTime = findex(times,0)
closestDepth = findex(depths,100)

# choose your own lat, long
iy_min, ix_min = get_closest_ij(latvals, lonvals, 50., -140)

# use units of original array, thus use
#print('%7.4f %s' % (east_vel[closestTime,closestDepth,iy_min,ix_min], east_vel.units))

''' TEST '''
# for any proper indices:
# (time,depth,latitude,longitude)

'''1st''' 

'''find given density at depth using equation''' 
def findPressure(depthUsed):
    return 1013.25 + (1025*9.81*depthUsed)/(10**4)

def findDensity(lat,long,depthUsed):
    pressure = findPressure(depthUsed)

    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)

    # sal at lat long depth
    salUsed = sal[0,depthFound,y_min,x_min]

    # temp at lat long depth
    tempUsed = temp[0,depthFound,y_min,x_min]

    return sea.dens(salUsed,tempUsed,pressure)

# sometimes calculation is -- idk why
# print(findDensity(25,25,0))

'''2nd'''
'''find given velocity at depth using data'''
def findVelocity(lat,long,depthUsed):
    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)
    horiz_v = east_vel[0,depthFound,y_min,x_min]
    vert_v = north_vel[0,depthFound,y_min,x_min]
    return [horiz_v,vert_v]

# vels = (findVelocity(25,25,0))
# print(vels[0],vels[1])

In [63]:
import random
'''3rd'''
'''Factor typical rising speed (mechanical,ballasts?), then slow with provisions,'''

# modify previous functions to include time component (which only matters if a day elapses)
def findDensity(lat,long,depthUsed):
    pressure = findPressure(depthUsed)

    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)

    # sal at lat long depth
    salUsed = sal[0,depthFound,y_min,x_min]

    # temp at lat long depth
    tempUsed = temp[0,depthFound,y_min,x_min]

    return sea.dens(salUsed,tempUsed,pressure)


# from sea currents
def findVelocity(lat,long,depthUsed):
    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)
    horiz_v = east_vel[0,depthFound,y_min,x_min]
    vert_v = north_vel[0,depthFound,y_min,x_min]
    return [horiz_v,vert_v]

# calculate the Δv using Δv = a * Δt
# and we are using intervals of 1 second
def updateVelocity(vel_x, vel_y, sea_vel_x, sea_vel_y):
    A = 3.6 * 2.4  # Cross-sectional area in m^2
    CD = 0.25  # Drag coefficient
    rho = 1025  # Density of seawater in kg/m^3
    m = 11000  # Total mass in kg
    delta_t = 1  # Time step in seconds

    # Calculate relative velocities
    rel_vel_x = vel_x - sea_vel_x
    rel_vel_y = vel_y - sea_vel_y

    # Calculate drag force based on relative velocity
    drag_force_x = 0.5 * CD * A * rho * rel_vel_x**2 * (-1 if rel_vel_x > 0 else 1)
    drag_force_y = 0.5 * CD * A * rho * rel_vel_y**2 * (-1 if rel_vel_y > 0 else 1)

    # Calculate acceleration due to drag
    accel_x = drag_force_x / m
    accel_y = drag_force_y / m

    # Update velocities with correct acceleration application
    vel_x += accel_x * delta_t
    vel_y += accel_y * delta_t

    return [vel_x, vel_y]

def print_elapsed_time(seconds):
    # Calculate hours, minutes, and seconds
    hours = seconds // 3600  # Divide by 3600 to get hours
    minutes = (seconds % 3600) // 60  # Get remainder from hours, divide by 60 for minutes
    seconds = seconds % 60  # Remainder from minutes are the seconds

    # Format the string to print out the time
    time_string = f"{hours} hour(s), {minutes} minute(s), and {seconds} second(s)"
    return (time_string)

# fix acceleration stuf!!!!! (divide by mass)
# determine ascension speed based on cases
# if boundary condition or oversunken then ...

# arbitrary points (if time)
# determined vel
'''Model Path using constant (then decaying speeds)'''
'''Factor typical rising speed (mechanical,ballasts?), then slow with provisions,'''
def simSub(v_x,v_y,v_z, x,y,z, riseSpeed, maxTime):

    # arbitrary amounts yet again
    # while ( oxygenated & powered )
    timer = 0


    print('initial x,y,z',x,y,z)
    curr_x = x
    curr_y = y
    curr_z = z

    curr_v_x = v_x
    curr_v_y = v_y
    curr_v_z = v_z

    while  (curr_z > 10):
        
        # check deadpoint

        #
        if curr_x == '--' or curr_y == '--':
            curr_x = random.uniform(15.1666,23.1666)
            curr_y = random.uniform(36.41668, 40.5)

        ### BOUNDARY CONDITIONS
        
        boundary = False
        if curr_y <= min(latvals) or curr_y >= max(latvals):
            print('lat outta bounds')
            boundary = True
        if curr_x <= min(lonvals) or curr_x >= max(lonvals):
            print('long outta bounds')
            boundary = True
        
        if(boundary):
            print('final x,y,z',curr_x,curr_y,curr_z,'seconds:',timer)
            return [curr_x, curr_y, curr_z]
        
        print(curr_x,curr_y)
        # find density

        # find accel
        
        sea_vel_x = findVelocity(*[curr_x,curr_y,curr_z])[0]
        sea_vel_y = findVelocity(*[curr_x,curr_y,curr_z])[1]

        curr_v_x = updateVelocity(curr_v_x,curr_v_y,sea_vel_x,sea_vel_y)[0]
        curr_v_y = updateVelocity(curr_v_x,curr_v_y,sea_vel_x,sea_vel_y)[1]

        # iterate on vel # 111,111 metres is a lat or long
        curr_x += curr_v_x/111111
        curr_y += curr_v_y/111111

        # upward movement !
        curr_z -= riseSpeed

        # repeat, think of cases???
        timer += 1
        
    print('final x,y,z',curr_x,curr_y,curr_z,'seconds:',timer)
    print('time elapsed:',print_elapsed_time(timer))
    print('meters travelled along latitude:',(x-curr_x)*111111)
    print('meters travelled along longitude',(y-curr_y)*111111)
   
    return [curr_x, curr_y, curr_z]




In [70]:
import random
min(depths)
max(depths)

# random.uniform(1, 10)
# then findex that
def helpSeed(low,high,arr):
    val = random.uniform(low,high)
    print(val)
    index = findex(val,arr)
    
    return index

def seedSubVals():
    latRand = random.uniform(15.1666,23.1666)
    longRand = random.uniform(36.41668, 40.5)
    randDepth = random.uniform(.49,800)
    rand_v_X = random.uniform(2,3)
    rand_v_Y = random.uniform(.5,1)
    rand_v_Z = 0

    
    riseSpeed = 0.3048*random.uniform(5,30)*(1/60)
    maxTime = 259200 # seconds in 3 days
    
    # simSub(v_x,v_y,v_z, x,y,z, riseSpeed, maxTime)
    return [rand_v_X,rand_v_Y,rand_v_Z,longRand,latRand,randDepth,riseSpeed,maxTime]




seeded = seedSubVals()
simSub(*seeded)
# randLat = 15.166 to 23.1666
# randLong = 36.41668 to 40.5
# randDepth = (.49) 4.9 e-01 to 1245
# rand_v_X = up to 3 kn (2-3)
# rand_v_Y = turning case, (irrelevant?) (.5 to 1)??? ***
# rand_v_Z =                                          ***

# simSub(v_x,v_y,v_z, x,y,z, riseSpeed)



# 4th
# Model Path using constant (then decaying speeds)
# Path: { Buoyancy, Drag, || Vectors}

# 5th, Simulations
# Error
# Carribbean Case Study?
# Cases
# Error Definition and Generalization
# Finally Visualization

# Close the dataset
# ds.close()

# distance in meters from initial
# time spent hours and minutes and seconds

initial x,y,z 40.04107080665146 18.780918594059592 631.3704328045694
40.04107080665146 18.780918594059592
40.04108798170371 18.78092662064815
40.04110185120716 18.780933926492718
40.04111356399906 18.78094063519159
40.04112373791295 18.780946840347784
40.04113275019984 18.780952614694524
40.0411408505518 18.780958015958245
40.041148213845204 18.780963090776677
40.04115496783101 18.78096787740409
40.041161208944686 18.780972407631292
40.04116701192328 18.780976708180837
40.04117243595384 18.78098080174186
40.04117752876569 18.780984707751426
40.041182329446336 18.78098844299398
40.04118687043432 18.78099202206767
40.041191178963786 18.780995457751814
40.041195278133536 18.78099876129968
40.04119918771282 18.781001942674294
40.04120292475836 18.78100501074005
40.041206504093736 18.781007973419822
40.04120993868655 18.781010837824784
40.041213239948625 18.781013610362468
40.041216417977495 18.78101629682731
40.04121948175243 18.781018902477012
40.04122243929503 18.781021432097297
40.04122

[40.04347616039026, 18.781479176490258, 9.928665577689499]

Results: 

Trial 1:
initial x,y,z 36.62247549198853 22.783140892170696 663.4158757286558
final x,y,z 36.6249487873822 22.783667236265007 9.977481632663594 seconds: 7442
time elapsed: 2 hour(s), 4 minute(s), and 2 second(s)
meters travelled along latitude: -274.8103244861321
meters travelled along longitude -58.482618662997396

Trial 2:
initial x,y,z 36.76977196725158 18.04569604129297 268.66669376354633
final x,y,z 36.771379421220054 18.046110712317752 9.921214437701105 seconds: 1904
time elapsed: 0 hour(s), 31 minute(s), and 44 second(s)
meters travelled along latitude: -178.6058178912446
meters travelled along longitude -46.07451223451439

Trial 3:
initial x,y,z 39.84072252244655 20.28923177731256 61.74360616337705
final x,y,z 39.84195534857907 20.289545867305367 9.97395619933734 seconds: 631
time elapsed: 0 hour(s), 10 minute(s), and 31 second(s)
meters travelled along latitude: -136.98054441068427
meters travelled along longitude -34.89885319101525

Trial 4:
initial x,y,z 39.57108254384413 18.593256190956478 97.90580530107084
final x,y,z 39.57266696364699 18.593592921237803 9.91949754842641 seconds: 772
time elapsed: 0 hour(s), 12 minute(s), and 52 second(s)
meters travelled along latitude: -176.04646871560877
meters travelled along longitude -37.41443828840983

Trial 5:
initial x,y,z 40.04107080665146 18.780918594059592 631.3704328045694
final x,y,z 40.04347616039026 18.781479176490258 9.928665577689499 seconds: 6830
time elapsed: 1 hour(s), 53 minute(s), and 50 second(s)
meters travelled along latitude: -267.26125927195704
meters travelled along longitude -62.28687445368442